## Epic fail, just gets stuck

In [ ]:
import gym
import ray

In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [ ]:
# Initialize Ray
ray.shutdown()
ray.init(ignore_reinit_error=True)

In [ ]:
print("Dashboard URL: http://{}".format(ray.get_webui_url()))

### Register MineRL Gym Environment to RLlib

In [ ]:
import minerl
from gym import envs

In [ ]:
### Try skipping the next 2 cells

In [ ]:
from minerl_rllib.envs.env import MineRLRandomDebugEnv

In [ ]:
# Register MineRL Gym Environment to RLLIB
# https://docs.ray.io/en/latest/rllib-env.html
from ray.tune.registry import register_env

def minerl_env_creator(env_config):
    import minerl
    
    if 'minerl_env_name' in env_config:
        # TODO use logger
        print('MineRL Env Name found...')
        env_name = env_config['minerl_env_name']
    else:
        # TODO use logger
        print('No MineRL Env name specified, using MineRLNavigateDense-v0')
        env_name = 'MineRLNavigateDense-v0'
        
        
#     # Check minerl environments are imported
#     # This only checks Gym environment not the ones registered in Ray!
#     all_envs = envs.registry.all()
#     env_ids = [env_spec.id for env_spec in all_envs]
#     print(env_ids)

# This doesnt work, need wrapper to discretize the action space for DQN
#     minerl_env = gym.make(env_name) 
    
    # Second version but has .nan rewards
#     core_env = gym.make(env_name) # A MineRLNavigate-v0 env
#     minerl_env = wrap_env(core_env, test=False)
    
#     minerl_env = gym.make('MineRLNavigateDense-v0')
    
    return minerl_env  

register_env("minerl", minerl_env_creator)

In [ ]:
from minerl_rllib.envs import register

In [ ]:
register()

In [ ]:
print('DONE')

In [ ]:
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.dqn import DQNTrainer

tune.run(PPOTrainer,
         config={"env": "MineRLRandomDebug-v0",
                 "use_pytorch": True,
                 'monitor':True, 
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 },
         stop={"training_iteration": 2, "timesteps_total": 50})  
# Config notes:
# "log_level": "INFO" for verbose,
# "eager": True for eager execution,